# Task
- [x] 静态方法和类方法，和静态属性
- [x] 装饰器，及语法特性探索
- [x] 插入排序
- [x] 希尔排序
- [x] 表达式和操作符重载

## Static v.s. Class

从下面的示例可以看出：

1. 实例方法第一个是`self`参数，类方法和静态方法没有
2. 类方法第一个参数是`cls`，静态方法没有
3. 用实例还是类名去调静态方法及类方法，效果是一样
4. `cls`关键字取到的属性是**当前类**，即如果有子类继承，那么子类在用`cls`取静态属性的时候就是子类自己的属性
5. 用类名取到的就是类名对应的类的静态属性

In [196]:
class Foo(object):
    x, y = 1, 2
    
    @classmethod
    def add(cls, *mixes):
        return mixes[0] + mixes[1]
     
    @classmethod
    def add2(cls):
        return cls.x + cls.y
    
    @classmethod
    def cls_or_static(cls):
        return Foo.x + Foo.y
    
    @staticmethod
    def add3():
        return Foo.x + Foo.y
    
class Bar(Foo):
    x, y = 5, 6
    
print('Bar.cls_or_static() is (1 + 2) = %d' % Bar.cls_or_static())

Bar.cls_or_static() is (1 + 2) = 3


In [191]:
foo = Foo()
bar = Bar()
print(Foo.add(3,4), Foo.add2(), Foo.add3(), Foo.cls_or_static())
print(Bar.add(3,4), Bar.add2(), Bar.add3(), Bar.cls_or_static())
print(foo.add(3,4), foo.add2(), foo.add3(), foo.cls_or_static())
print(bar.add(3,4), bar.add2(), bar.add3(), bar.cls_or_static())

7 3 3 3
7 11 3 3
7 3 3 3
7 11 3 3


## Decorator

In [53]:
# 装饰器不带参数
def wrap1(func):
    print("step 1, 进入装饰器函数体")
    def impl(*args, **kwargs):
        print("step 2，进入装饰器实现，得到参数:", args, kwargs)
        return func(*args, **kwargs)
    return impl

@wrap1
def func1(x, y):
    print("step 3，函数体实现")
    return x+y

    
func1(3,4)

step 1, 进入装饰器函数体
step 2，进入装饰器实现，得到参数: (3, 4) {}
step 3，函数体实现


7

In [55]:
# 装饰器带参数
def wrap2(*args, **kwargs):
    print("step 1, 最外层，传入装饰器参数：", *args, **kwargs)  # kwargs是得不到的，多参数只能靠解包args
    def inner(func):
        print("step 2, 第二层，传入被装饰函数", func)
        def impl(*args, **kwargs):
            print("step 3，第三层，得到函数参数", args, kwargs)
            return func(*args, **kwargs)
        return impl
    return inner

@wrap2({"route":"/api/user", "name":"walker"})
def func2(a, *args, **kwargs):
    print("step 4, 被装饰函数体实现")
    
func2("hello",orderid=1234)

step 1, 最外层，传入装饰器参数： {'route': '/api/user', 'name': 'walker'}
step 2, 第二层，传入被装饰函数 <function func2 at 0x7fbb532469e0>
step 3，第三层，得到函数参数 ('hello',) {'orderid': 1234}
step 4, 被装饰函数体实现


### 装饰器语法思考

如果装饰器只包一个函数的话，能不能直接返回这个函数而不是多包一层呢？

**可以的**。见下方示例

但是，仔细观察上述示例，会发现
- 传入被装饰的函数
- 给装饰器传参
- 给被装饰的函数传参（其实是读取）

每一件事都需要写一个方法来包一层，传入想要的东西，所以：

- 如果你只是单纯包一下，可以直接返回函数
- 如果你的被装饰函数（`func`）有参数，那么需要在外层传入`func`，里层传入参数，见示例1
- 如果你的装饰器本身也需要传参，那么得最外层得再包一层用来接参数，第二层接`func`，第三层接`func`的参数

所以这么奇怪的语法只不过是python怎么支持你去传递上述三个对象而已

In [52]:

def wrap3(func, *args, **kwargs):
    print('step1,', args, kwargs)  # 拿不到函数参数
    return func

@wrap3 # 这种写法既不能给装饰器传参，也无法接到func的参数
# @wrap3(a="1",b=2) # 这种写法不会默认传入func3
def func3(a,b,*args,**kwargs):
    print("step 2, enter func3 body")

func3("a",6,"hello", name="walker")

step1, () {}
step 2, enter func3 body


## insert sort

总体思路是每个新数字都在已经排好序的部分里面找到**第一个**比它大的，然后添加（插入）到它前面，如果找不到，说明它自己就是最大的。     
做了如下五种实现，其中：

- 前两种是用一个数组来存遍历过程中排序好的数组，
- 第三个是用了每次只比较第一个和最后一个，要么添加到最前，要么添加到最后，不满足则递归，已经不算插入排序了吧
- 第四，五个是不断改变字符串本身，而不另存数组

说明：

1. 没有边界压缩，每次都遍历整个数组，但是用了语法特征让代码量相当少
2. 与1的区别在于把压缩语法展开了，这样在找到第一个大于当前数的时候就能退出本次循环
3. 修改过程中的附产物，不属于插入排序，略，而且效率不高
4. 把当前数放到左边已排好序的合适位置（第一个大数前），用了for循环
5. 同4，用了while循环

执行两万个随机数排序的效果：
```
1: 17.83499026298523
2: 7.970043897628784
3: 超时
4: 9.292963027954102
5: 21.950623989105225
```

可见：

1. 方法**2**和方法**4**效率差不多，2略优于4，可见改变数组比改变字符串消耗要小很多
2. 方法**5**只是把方法**4**的`for`换成了`while`，居然性能差这么多


In [17]:
def insert_sort(arr):
    rst = arr[0:1]
    for i in arr[1:]:
        large = [idx for idx, itm in enumerate(rst) if itm > i]
        index = len(rst) if not large else large[0]
        rst.insert(index, i)
    return rst

def insert_sort2(arr):
    rst = arr[0:1]
    for i in arr[1:]:
        found = False
        for idx, j in enumerate(rst):
            if j > i:
                rst.insert(idx, i) # 排到第一个比它大的前面
                found = True
                break;
        if not found:
            rst.append(i)
    return rst

def insert_sort3(arr, n=0):
    '''
    这次不操作一个新的数组，每次改元素位置
    n 表示已经排好了序的最大索引（个数-1），默认0，即只排了一个
    '''
    # 先排除例外，如果只有3个元素，已经排好了两个，直接插到最中间就好了
    if len(arr) == 3 and n == 1:
        arr[1], arr[2] = arr[2], arr[1]
        return arr
    for i in arr[n+1:]:
        # 如果当前位置的数比排好序的最后一个数大，不动
        if i > arr[n]: pass 
        elif i <= arr[0]:
            # 如果比最小的小，那么移到最前面来
            del arr[n+1]
            arr.insert(0, i)
        else:
            # 这里不用逐个比较，试试迭代，不断复用前面的比较头尾的方法。
            right = arr[n]
            sub_sort = insert_sort3(arr[1:n]+[i], n-2)
            arr[1:n+2] = sub_sort + [right]
        n += 1
    return arr

def insert_sort4(arr):
    n = 1 # 已经排好了序的数量，默认1，即只排了一个
    for i, item in enumerate(arr[0:]):
        if i==0: continue # 第一个数不需要比较
        for j, el in enumerate(arr[0:n]):
            if el >= item:
                del arr[i]
                arr.insert(j, item)
                break
        n += 1
    return arr

arr = [12, 9, 2, 4, 0, 12, 9, 5, 25, 43, 1, 6]

insert_sort(arr.copy()), insert_sort2(arr.copy()), insert_sort3(arr.copy()), \
insert_sort4(arr.copy()), insert_sort5(arr.copy())

([0, 1, 2, 4, 5, 6, 9, 9, 12, 12, 25, 43],
 [0, 1, 2, 4, 5, 6, 9, 9, 12, 12, 25, 43],
 [0, 1, 2, 4, 5, 6, 9, 9, 12, 12, 25, 43],
 [0, 1, 2, 4, 5, 6, 9, 9, 12, 12, 25, 43],
 [0, 1, 2, 4, 5, 6, 9, 9, 12, 12, 25, 43])

## operator overloading

昨天已经做过了，但今天增加了新内容（`simplify`），可以让表达式打印出来更简洁。

In [72]:
class Expr(object):
    def eval(self, **values):     # evaluate the expression
        pass
    def deriv(self, x):           # get the derivative of x
        pass
    
    def __add__(self, other):     # overloading + operator
        return Add(self, other).simplify()
    def __sub__(self, other):     # overloading - operator
        return Sub(self, other).simplify()
    def __mul__(self, other):     # overloading * operator
        return Mul(self, other).simplify()
    def __neg__(self):            # overloading - operator(单目)
        return Neg(self)
    def __truediv__(self, other): # overloading / operator
        return TrueDiv(self, other).simplify()
    
class Const(Expr):
    def __init__(self, value):
        self.value = value
    def eval(self, **values):
        return self.value
    def deriv(self, x):
        return Const(0)
    def __repr__(self):
        return str(self.value)
    
def get_var_name(x):
    return x.name if isinstance(x, Variable) else x
    
class Variable(Expr):
    def __init__(self, name):
        self.name = name
    def eval(self, **kwargs):
        '''
        x.eval(x=3, y=4) ==> 3
        x.eval(y=4) ==> error
        '''
        if self.name in kwargs:
            return kwargs[self.name]
        else:
            raise Exception(f"Variable {self.name} is not found")
    def deriv(self, x):
        '''
        derivative of var itself is 1, 
        otherwise is 0
        '''
        var_name = get_var_name(x)
        return Const(1 if var_name == self.name else 0)
    def __repr__(self):
        return self.name
    
x1 = Variable("x")
print(x1)
print(x1.eval(x=3))
print(x1.deriv("x"))
print(x1.deriv("x").eval(x=4))
print(x1.deriv("y").eval(x=4))

x
3
1
1
0


In [73]:
class Add(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) + self.right.eval(**kwargs)
    def deriv(self, x):
        # get a NEW Add object
        return self.left.deriv(x) + self.right.deriv(x)
    def __repr__(self):
        return f"({self.left} + {self.right})"
    
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            return Const(left.value + right.value)
        return self

class Sub(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) - self.right.eval(**kwargs)
    def deriv(self, x):
        return self.left.deriv(x) - self.right.deriv(x)
    def __repr__(self):
        return f"({self.left} - {self.right})"
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            return Const(left.value - right.value)
        return self
    
class Neg(Expr):
    def __init__(self, value):
        self.value = value
    def eval(self, **kwargs):
        return -self.value.eval(**kwargs)
    def deriv(self, x):
        return -self.value.deriv(x)
#     def simplify(self):
#         if isinstance(self.value, Const):
#             return Const(-self.value.value)
#         return self
    def __repr__(self):
        return '(-%s)' % self.value
        
class Mul(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) * self.right.eval(**kwargs)
    def deriv(self, x):
        '''
        (uv)' = u'v + v'u  (Chain Rule)
        '''
        u, v = self.left, self.right
        return u.deriv(x) * v + v.deriv(x) * u
    def __repr__(self):
        return f"({self.left} * {self.right})"
    
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            return Const(left.value * right.value)
        if left_const:
            if left.value == 0: return Const(0)
            if left.value == 1: return right
        if right_const:
            if right.value == 0: return Const(0)
            if right.value == 1: return left
        return self
    
class TrueDiv(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) / self.right.eval(**kwargs)
    def deriv(self, x):
        '''
        (u/v)' = (u'v - v'u) / v^2 (Quotient Rule)
        '''
        u, v = self.left, self.right
        return (u.deriv(x)*v - u*v.deriv(x))/(v*v)
    def __repr__(self):
        return f"({self.left} / {self.right})"    
    
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            if right_const.value == 0:
                raise Error("divide by zero error")
            return Const(left.value / right.value)
        if left_const and left.value == 0: return Const(0)
        if right_const and right.value == 1: return left
        return self
    
c1 = Const(13)
c2 = Variable("a")
c = c1 + c2
d = c1 - c2
e = c1 * c2
f = c1 / c2
g = -c1
h = -c2
print(f'{c} = {c1.eval()} + 5 = {c.eval(a=5)}')
print(f'{d} = {c1.eval()} - 5 = {d.eval(a=5)}')
print(f'{e} = {c1.eval()} * 5 = {e.eval(a=5)}')
print(f'{f} = {c1.eval()} / 5 = {f.eval(a=5)}')
print(f'{g} = {g.eval(a=2)}')
print(f'{h} = {h.eval(a=2)}')
print('-'*24)
print(f'{c}\' = {c.deriv("a").eval(a=3)}')
print(f'{d}\' = {d.deriv("a").eval(a=3)}')
print(f'{e}\' = {e.deriv("a").eval(a=3)}')
print(f'{f}\' = {f.deriv("a").eval(a=3)}')


(13 + a) = 13 + 5 = 18
(13 - a) = 13 - 5 = 8
(13 * a) = 13 * 5 = 65
(13 / a) = 13 / 5 = 2.6
(-13) = -13
(-a) = -2
------------------------
(13 + a)' = 1
(13 - a)' = -1
(13 * a)' = 13
(13 / a)' = -1.4444444444444444


求$(2*x-6y)/(3*x+4*y)$在x=2, y=3处的的导数值

In [74]:
x = Variable("x")
y = Variable("y")
exp1 = Const(2) * x - Const(6) * y
exp2 = Const(3) * x + Const(4) * y
exp = exp1 / exp2
d1 = exp.deriv(x)
d2 = exp.deriv(y)
values = {"x":2, "y":3}
print(exp)
print(f'∂f/∂x = {d1}')
print(f'∂f/∂y = {d2}')
print(values,'----'*10)
print(f'∂f/∂x = {d1.eval(**values)}')
print(f'∂f/∂y = {d2.eval(**values)}')

(((2 * x) - (6 * y)) / ((3 * x) + (4 * y)))
∂f/∂x = (((2 * ((3 * x) + (4 * y))) - (((2 * x) - (6 * y)) * 3)) / (((3 * x) + (4 * y)) * ((3 * x) + (4 * y))))
∂f/∂y = (((-6 * ((3 * x) + (4 * y))) - (((2 * x) - (6 * y)) * 4)) / (((3 * x) + (4 * y)) * ((3 * x) + (4 * y))))
{'x': 2, 'y': 3} ----------------------------------------
∂f/∂x = 0.24074074074074073
∂f/∂y = -0.16049382716049382


## 希尔排序

每隔几个形成一个数组来排序，避免某些宏观上倒序的数据会造成大量的**长距离**交换操作，希尔排序是初始以length/2为步长的，也就是说最长距离的交换也只可能有最长纯插入排序交换距离的一半。

几个要注意的地方：

1. 希尔排序里每组的插入排序比单纯一个数组的插入排序效率要低，因为后者是真插入，而希尔是为了实现“插入”的**效果**，把大数从右边一个个地往左挪，代码上并不是真的直接插入，因为如果直接插入，其它组的序号就全变了
2. 为了实现从右到左跟已排序的数组做比较，左边的数组就得是反序的，索引0其实已经是最后一个已排序的索引，要取得正序上的索引（用来做元素替换），需要用已排序的数组长度（其实就是当前比较的元素的索引）减去反序的索引
3. 所以很多人看示例代码有个类似`i-j`的操作，如果是自己去写的话，会发现这个问题，自己想办法把正序算出来，而如果直接看别人代码，是比较难理解这里是在转换数组索引的操作
4. 即使实现了，执行20000条随机数组排序仍然很慢（达到了几分钟），不知道是不是我实现的问题
5. 网上的标准方法不是用"反序比较“的思路，而是当前元素跟前一个step的元素比，所以不存在换索引的问题，但是执行效率怎么高那么多。是不是数组的反向切片方法本就很消耗？

In [22]:
def i_sort(arr, start, step):
    '''
    从右向左两两互换把大的交换到右边，才能实现把当前元素“插入”目标位置
    比直接一排数字用list.insert(index, element)要繁琐一些（时间，空间复杂度都增加？）
    因为不能改变其它组的元素，只能两两互换
    '''
    for index, right in enumerate(arr):
        # 排除第一个，和不属于本组的元素
        if index % step or index == 0: continue
        # 用index来表示每一轮排序和非排序的边界，
        # 因为左边排序部分我们是倒序搜索的，需要用这个index减去反序时的索引得到正序索引
        i = index 
        for j, left in enumerate(arr[i::-1]):
            # 排除第一个，和不属于本组的元素
            if j % step or j == 0: continue
            j = index - j # 反序索引变正序索引
            if left > right:
#                 arr[i], arr[j] = arr[j], arr[i] # 这里把对小数的赋值放到循环出来后，减少消耗
                arr[i] = left
                i = j
        arr[i] = right       

def shell_sort(arr):
    gap = len(arr) // 2
    while gap >= 1:
        for i in range(gap):
            i_sort(arr, i, gap)
        gap = gap // 2
    return arr

def shell_sort2(arr):
    group = len(arr) // 2
    
    while group > 0:
        for i in range(group, len(arr)):
            right   = arr[i]
            current = i
            while current >= group and arr[current - group] > right:
                arr[current] = arr[current - group]
                current -= group
            arr[current] = right
        group //= 2
    return arr

arr = [2, 15, 3, 22, 7, 0, 1, 6, 39, 9, 12, 3, 45, 17, 25, 17, 33, 6]

import numpy as np
import time
np.random.seed(7)
length = 20000
arr = list(np.random.randint(0, 10000, size=(length,)))
# start = time.time()
# s1 = shell_sort(arr)
# print("shell: %.18f" % (time.time() - start))  # 10000条用了75秒
start = time.time()
s1 = insert_sort2(arr)
print("insert2: %.18f" % (time.time() - start))
# start = time.time()
# s2 = insert_sort4(arr)
# print("insert4: %.18f" % (time.time() - start))
start = time.time()
s3 = shell_sort2(arr)
print("shell2: %.18f" % (time.time() - start))
# print(f"s1:{s1[:50]}\ns2:{s2[:50]}\ns3:{s3[:50]}")
# print(f"s2:{s2[:50]}\ns3:{s3[:50]}")
print(s3[:50])

insert2: 7.337036848068237305
shell2: 0.151087760925292969
[0, 0, 1, 1, 1, 3, 3, 4, 5, 6, 6, 7, 7, 8, 10, 10, 11, 11, 12, 13, 13, 13, 13, 14, 14, 14, 14, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 20, 20, 21, 22, 22, 22, 23, 24, 25, 25]


In [32]:
start = time.time()
m = 0
for i in range(5000):
    for j in arr[::2]:
        m = i + j
print("->", time.time()-start)
start = time.time()
m = 0
for i in range(5000):
    for j in arr[-1:0:-2]:
        m = i + j
print("<-", time.time()-start)

-> 16.352194786071777
<- 16.24852228164673


In [30]:
a=[1,2,3,4,5,6,7,8,9,0]
a[-1:0:-2]

[0, 8, 6, 4, 2]

In [2]:
for i in range(10):
    print(i)
    i += 2
    print(">>", i)

0
>> 2
1
>> 3
2
>> 4
3
>> 5
4
>> 6
5
>> 7
6
>> 8
7
>> 9
8
>> 10
9
>> 11
